In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')


In [ ]:
#!pip install treelib
#!pip install text_to_image
#!pip install pystemmer 

In [ ]:
pip install pystemmer

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import os
import random
import time

%matplotlib inline
import matplotlib.pyplot as plt, time
from mpl_toolkits.mplot3d import Axes3D

from sklearn.datasets import load_files
from sklearn.externals import joblib #in place of pickling, its better
from sklearn.decomposition import PCA  #use for similarity anlysis plots
from sklearn.manifold import TSNE #use for similarity analysis plots
import seaborn as sns


# NLP Packages
import scipy.io as spio
import spacy
from scipy import ndimage
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm")
from spacy import displacy #this allows the visualizations to be printed in jupyter

 #py stemmer is a better implementation of snowball than nltk
import Stemmer
stemmer = Stemmer.Stemmer('english')

import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer() # Use the wordnet corpus to define the lemma
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import NOUN

import xml.dom.minidom
from IPython.display import HTML
import re #regular expression tokenizer 

import markovify #Markov Chain Generator

# Neural Network Packages
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import text_to_word_sequence

from itertools import chain  #flatten list of list

from nltk.probability import FreqDist

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# USER CONFIGURATION PARAMETERS

In [ ]:
## SELECT CORPUS TO USE IN ANALYSIS
corpusname='lyrics'  # OPTIONS: lyrics, cbt, movies
corpussize= 'all'  # number or "all"
base = 'stem'   #'lemma'  or 'stem'  This also selects which base to use in the corpus cleaning for consist

new_model=True 

# Read in Raw Data
## Read in Affect Word Lists
<a id='readinaffect'></a>

In [ ]:
from treelib import Node, Tree
#Read in Wordnet Affect

def readxml(filename):
    #use the parse() function to load and parse the xml file
    doc = xml.dom.minidom.parse(filename)
    
    #print out the document node and the name of the first child tag
    print(doc.nodeName)
    print(doc.firstChild.tagName)
    
    #get a list of xml tags from the document and print them
    categ = doc.getElementsByTagName("categ")
        
    data = []
    for c in categ:
        n=c.getAttribute("name")
        #print(n)
        p=c.getAttribute("isa")
        data.append([n,p])
    return(data)


#emthyarch= readxml('../DataSets/wordnet-affect/wn-domains-3.2/wn-affect-1.1/a-hierarchy.xml')
emthyarch = readxml('../input/wordnetaffect/a-hierarchy.xml')
effdf = pd.DataFrame(emthyarch, columns=['Emotion','Parent'])
effdf=effdf.drop([88,266,277,300,305,307]) #these are all loops the parent and child are same
#wnemots =effdf['Emotion'].tolist()

# Create tree object
tree = Tree()
tree.create_node('root','root') #root
#tree.create_node("Joy", "joy",parent='positive')

for i, r in effdf.iloc[1:].iterrows():
    #print (i,'Emotion', r["Emotion"],'Parent', r["Parent"])
    tree.create_node(r['Emotion'],r['Emotion'],parent=r['Parent'])

pos_t = tree.subtree('positive-emotion')
neg_t = tree.subtree('negative-emotion')
amb_t = tree.subtree('ambiguous-emotion')
emtwn = tree.subtree('emotion')

print('Here is an example of the positive list under the calmness branch:\n')
pos_t.subtree('calmness').show()
print(emtwn.show())

wnemots=(','.join([emtwn[node].tag for node in \
            emtwn.expand_tree(mode=Tree.DEPTH)]))
wnemots=wnemots.split(',')
print(wnemots)


#print('\nLets look at the tree where our classifed word is:')
#tree.subtree('emotion').subtree('gloom').show()
#tree.show()
#print(','.join([tree[node].tag for node in tree.rsearch('sad')]))


In [ ]:
# 28 Russell terms
russell=['happy','delighted','excited','astonished','aroused','tense','alarmed','angry','afraid','annoyed','distressed','fustrated','misearable','sad','gloomy','depressed','bored','droopy','tired','sleepy','calm','relaxed','satisfied','at ease','content','serene','glad','pleased']
# labels used in the images
# note: 'Having Cold' was changed to cough
ilabels=['Anger','love','Fear','Failing','Guilt','Anxiety','Stress','Exhaustion','Tiredness','Mannia','Defecation','Heartbeat','Movement','Pride','Relaxation','Succeeding','Wanting','Orgasm','Craving','Feeling touch','Social Longing',
        'Hositity','Losing','Shame','Despair','Nervousness','Shriving','Itching','Numbness','Acceleration','Eating','Breathing','Smelling','Winning','Laughing','Happiness','Pleasure','Love','Sympathy','Sexual Arousal','Lounging for',
       'Despise','Disgust','Disappointment','Sadness','Lonliness','Aching','Flu','Coughing','Hotness','Thirst','Satiation','Tasting','Sleeping','Dazzled','Daydreaming','Gratefulness','Self-regulation','Surprise','Closeness','Togetherness',
       'Ostractism','Panic','Social exclusion','Feeling pain','Depression','Toothache','Fever','cough','Coldness','Hunger','Urination','Hearing','Vitality','Imagining','Thinking','Being Conscious','Attending','Memorizing','Remembering','Forgetting',
       'Drunkenness','Hangover','Suffocation','Headache','Nauseous','Stomach flu','Heartburn','Dizziness','Sneezing','Sweating','Drinking','Seeing','Speaking','Reading','Reasoning','Inferring','Estimating','Recognition','Recollection','Waiting']

emotwords = wnemots + russell + ilabels  #Join all my emotion words together so that the model is filtered by all of them # or emots
print('There are ',len(emotwords),'emotional words in our emots list made up from Russells 28 original, the labels from the 100 body sensation maps and the ',len(wnemots),'affect wordnet terms\n',emotwords[:20]) 
#some functions use emotwords - some use emots - need to clean up



### Clean Affect Word Lists

In [ ]:
## CLEANING FUNCTIONS

def unique(list1):    # function to check for duplicates
    list_set = set(list1)   # insert the list to the set; SETS can only have unique values
    unique_list = (list(list_set))  # convert the set back to a list 
    return(unique_list)


def get_wordnet_pos(word):
    #"""Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wn.ADJ,
                "N": wn.NOUN,
                "V": wn.VERB,
                "R": wn.ADV}
    return tag_dict.get(tag, wn.NOUN)  #was wn.NOUN

def lemmaemots(word):
    #print('PRE LEMMA: ',word)
    cleanemots= wnl.lemmatize(word, get_wordnet_pos(word))  #wordnet lemmetizer
    #print('POST LEMMA:',cleanemots)
    #cleanemots = cleanemots.lower()
    return cleanemots


def stememots(emotwords):   #read in a list of words
    #print('PRE STEM: ',word)
    cleanemots = [stemmer.stemWord(word) for word in emotwords]  #make a list of stemmed words
    cleanemots = [cw.lower() for cw in cleanemots]  # make a list of lower case stemmed words
    cleanemots=unique(cleanemots)  # return a list of just unique stemmed lowercase words
    #print('POST STEM:',cleanemots)
    return cleanemots  #this is a list

lemots=[lemmaemots(word) for word in emotwords]
lemots=unique(lemots)


stemots=stememots(emotwords)


#print('\nRAW:',len(emotwords), emotwords[:10])
#print('\nLEMMA & UNIQUE:',len(lemots), lemots[:10])
#print('\nSTEMMED & UNIQUE:',len(stemots))

# SELECT LEMMA OR STEM
if base=='lemma':
    cleanemots=lemots
else:
    cleanemots=stemots

#cleanemots=cleanemots[0]  #get rid of list of list for no reason

print('\nI have ',len(stemots),"in the STEMMED & UNIQUE list.  The",base,"list has been selected. \nHere's an example:")
print(cleanemots[:10])


APPEND 'IN' WORDS TO STEMMER
There are a lot of words in lyrics that are not real "lovin" "diein" to catch these append the stemmer & lemmer to recongize them 

Lets see if using the stemed words provide a bit more clustering, we'll have to use stem in the corpus too

## Read in Corpus Data
use this to select which corpus data to read in

### Children's Book Test Corpus
We'll read in a corpus to train our similarity analysis on

In [ ]:
## Read in Childrens book corpus
if corpusname == 'cbtnm':
    
    f = open(r"../input/childrensbooks/cbt_train.txt","r")

    cbt = f.read()
    print(type(cbt),'example',corpus[:30])

    cbtnm = 'cbt'

    # split into sentences
    #sentences = sent_tokenize(corpus) # or f if cbt
    #sentences = unique(sentences) #unique sents NOT SURE I WANT TO DO THIS
    print('There are ',len(corpus),'characters in the',corpusname,' corpus')
    print('There are ',len(sentences),'sentences in the',corpusname,' corpus')

    #print(type(sentences))
    #sentences[:5]
    corpus=cbt
else: print('CBT Corpus Not Selected')

In [ ]:
# Movie Corpus
if corpusname == 'movies':
    f = open(r'../input/movie-dialog-corpus/movie_lines.tsv')
    movies=f.read()

    print(type(cbt),'example',corpus[:30])

    movnm = 'movies'

    # split into sentences
   # sentences = sent_tokenize(movies) 
   # sentences = unique(sentences) #unique sents NOT SURE I WANT TO DO THIS
   # print('There are ',len(movies),'characters in the',movnm,' corpus')
   # print('There are ',len(sentences),'sentences in the',movnm,' corpus')

    #print(type(sentences))
   # sentences[:5]
    corpus=movies
else: print('Movie Corpus Not Selected')

### Song Lyrics Corpus
So the CBT corpus is not as valuable as I thought it would be.  It turns out the language used is sort of odd, and doesn't use as many emotional words as I thought it would.  

Songs are full of emotion! Lets use that data set. 

In [ ]:
songdata=[]
if corpusname == 'lyrics':
    songdata = pd.read_csv("../input/150k-lyrics-labeled-with-spotify-valence/labeled_lyrics_cleaned.csv")
    lyrics=songdata['seq']  #returns a list of lyrics 1 item per song
    lyricsnm='spotify'
    corpus = lyrics  
    print(type(corpus),'\nExample of corpus:\n',lyrics[:5])

else: print('Song Lyrics Corpus Not Selected')


In [ ]:
print('************************ MY SELECTED CORPUS IS ',corpusname,'*******************************')
#print(corpus[0])
print(type(corpus))
print(len(corpus))
if corpussize=='all':
    corpussize=len(corpus)
    print(corpussize)
corpus[:5]

### Clean Corpus

#### CLEANING
* STOP WORDS - not removed because the downsampling of frequent words will get rid of them anyway - the sample parameter = 0.001 is the pruner.  It may give better results with since we are looking for neighbors for context.  
* LOWER CASE -  since i don't expect there to be too many proper names (apple vs Apple) or acrynoms in my corpus I'll strip caps  
* PUNCTUATION -  I dont care about punctuation except at end of sentences; the end of sentence punctuation will be used to split by sents.   
* LEMMA - NLTK wordnet lemmetizer takes input string; if you pass a word it'll treat it like a noun; to lemma a sentence/lyrics/pharse: 1st convert sent to touples with word and pos;  use the converter to account for the pos by nltk.pos_tag() and what's expected by wordnetLemmetizer().  For lyrics the entire song is sent - for movies would need to convert to sent lemma. 




### Emotional Based Corpus
Next, we limit the corpus to only include sentences which have at least one emotional word it.  We use the combined all emots list to filter the corpus by



In [ ]:
s=set(cleanemots)
s

In [ ]:
import spacy

def nltk2wn_tag(nltk_tag):  #accounts for diffences in nltk tagging vs wordnet.lemmetizer
    if nltk_tag.startswith('J'):
        return wn.ADJ
    elif nltk_tag.startswith('V'):
        return wn.VERB
    elif nltk_tag.startswith('N'):
        return wn.NOUN
    elif nltk_tag.startswith('R'):
        return wn.ADV
    else:                    
        return None

def findemotlines(text):  #use this type of function to limit each line with emotional terms
    emotlines=[]
    st=text.splitlines()        #SPLIT LYRIC INTO LINES
    #print('ST: ',st)
    #print('CLEANEMOTS: ',cleanemots[:5],len(cleanemots))
    for line in st:                    # evaluate each sentence in the cleansents list - this loops through the cleansents list for each emotion 
        #CHECK IF LINE HAS WORD IN CLEAN EMOTS.
        x=set(line.split()) & set(cleanemots)   #use set because word list is long and need to do this operation alot
        if x!=set():  # if it has an emotional word grab the entire line
            emotlines.append(line)              
    return(" ".join(emotlines))

def lemmatize_sentence(sentence):
    print('*** TIME TO LEMMA ***')
    tokens=regexp_tokenize(sentence,pattern='\s+',gaps=True)  #use REGEXP_TOKENIZER - ITS FASTER THAN WORD_TOKENIZER
    nltk_tagged = nltk.pos_tag(tokens)    #tokenize then tag
    wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)  #translates between nltk & wn tags
    lemma_words = []
    for word, tag in wn_tagged:
        #print(word, tag)
        if tag is None:                        
          lemma_words.append(word)
        else:
          lemma_words.append(wnl.lemmatize(word, tag))  #replace word with lemma word using the wn tags
    #return " ".join(lemma_words)
    return lemma_words

def clean_text(text,emotional=False):   
    print('>>>>>>>>>>>>>>>>> CLEANING ONE LYRIC AT A TIME <<<<<<<<<<<<<')
    # change all in' to ing as in lovin' to loving
    #print(text)
    text=text.replace("in'", "ing")
    #print(text)
    #ONLY keep EACH LINE if has emotional word in it -> emotionally strong corpus
    print('\n\nbefore:',len(text),'\n')
    if emotional:
        #print('check to see if any of these words are in my lyric:')
        text= findemotlines(text) 
    print('AFTER: ',len(text),'\n')  
    
    #replace new line & carriage retuns with space
    text=text.replace("\n", " ").replace("\r"," ")
    #print('new line',text)
    
    #replace numbers and punctuation  with space except '  # can do this during tokeniization in lemma??
    punc_list = '"?!#$%&().*+, -./:;<=>@[\]^_`{|}~' + '1234567890'
    t=str.maketrans(dict.fromkeys(punc_list," "))
    text=text.translate(t)

    
    #replace single quotes wtith empty set 
    t=str.maketrans(dict.fromkeys("'",""))
    text=text.translate(t)

    #make all lower caps
    text = text.lower() 
    
    # lemma each lyric - maybe want to split by sentence: 
    if base=='lemma':
        tokens = lemmatize_sentence(text)
    elif base=='stem':
        tokens=regexp_tokenize(text,pattern='\s+',gaps=True) 
        tokens = [stemmer.stemWord(word) for word in tokens]  
    #print('LENGTH OF TOKENS', len(tokens))
    return tokens

clean_text("I be lovin' these cakes!\nlovin' lovin' wantin' more cake and coffee!!\nOh yea more cake and coffee!",True)

In [ ]:
print('CORPUS SIZE',corpussize)
print('STEM or Lemma:', base)
cleancorp=[]
tokens=[]

for corpitem in corpus[:corpussize]:
        #print('*****CORPUS ITEM ******:\n ',corpitem[:50])
        tok=clean_text(corpitem, emotional=True) #call tokenizer
        text=" ".join(tok)
        print('>>>>> CLEANED TEXT: >>>>',text[:50])
        print('>>>>> CLEANED TOKENS: >>>>',tok[:50])
        cleancorp.append(text)
        tokens.append(tok)

#print('CLEAN - STEMMED AND TOKENIZED ')  #nltk regexp is way faster!
#print(len(cleancorp))
#print('example of cleaned corp\n',cleancorp[:3])
#print('\nexample of tokens\n',tokens[:3],'\nTOKEN LENGTH:',len(tokens))  #list of lists

alltokens=list(chain.from_iterable(tokens))  #single list of all tokens
print('alltokens has',len(alltokens), ' and look like this: [token1,token2,token3,token4,...]')

print('\CLEANED EMOT WORDS',type(cleanemots), len(cleanemots),'\n')

#Join the emotionwords with the corpus list so model has every emotional words
# +, extend, append, Insert, [list1,list2} and #tokens[len(tokens):len(tokens)] = emotwords  didn't work - lets do it mannually
tok_emots=[]
for i in tokens:
    tok_emots.append(i)
tok_emots.append(cleanemots)  #emots are cleaned
print('New List to model on length',len(tok_emots), '\nIt looks like this: [[lyric1],[lyric2],[lyric3],[],[],[emotwords]] where each lyric is tokenized [word1,word2,word3]')

### Frequency Distribution of Tokens

In [ ]:
fdist = FreqDist(alltokens)
#print(fdist)

print(fdist)
import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

# Simialarity Analysis
## Word2Vec Model
Latent Semantic Analysis (LSA) is the most popular technique of Corpus-Based similarity. LSA assumes that words that are close in meaning will occur in similar pieces of text.  One type of LSA is Word2vec, a word embedding model, originally published by \cite{Milkov} is 2-layer neural network vectorizes words from a corpus and returns a set of vectors representing the words.  Word2vec is able to discern meanings of words by the context in which they are used. There are two methods used, either a continuous bag of words or a continuous skip-gram algorithm.  

Great internal what its doing reference: http://jalammar.github.io/illustrated-word2vec/

In [ ]:
# Load the model
import gensim, os
from gensim.models.word2vec import Word2Vec

new_model=True      # new_model=True runs a new model; new_model=False loads the most recent kaggle version
# Using params from Word2Vec_FastText_Comparison
params = {
    'alpha': 0.025,  # alpha (float, optional) – The initial learning rate.
    'size': 25,     #The size of the dense vector to represent each token or word (i.e. the context or neighboring words
    'window': 5,     #The maximum distance between the target word and its neighboring word.
    'iter': 5,       #Number of iterations (epochs) over the corpus
    'min_count': 1,  #Minimum frequency count of words. The model would ignore words that do not satisfy the min\_count.
    'workers': 2,     #,   #How many threads to use behind the scenes
    'sg': 1,        # sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
#    'hs': 0,        # hs ({0, 1}, optional) – If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
#    'negative': 5   # negative (int, optional) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.

}

if new_model:
    model = Word2Vec(tok_emots, **params)  # wordlist token list
    # Save the entire model as a SavedModel in HDF5 format.
    model.save('AffectSimialirty_Model.h5') 
    # To export our model as text just to inspect it 
    model.wv.save_word2vec_format('wvmodel_vectorsT.txt', binary=False)
    print('New Model Created and saved')
    print(model)
if not new_model:
    # Load the Model back from file
    print("loading model...")
    model = gensim.models.Word2Vec.load('../input/affectsimilarities/AffectSimialirty_Model.h5')
    print(model)
    print('complete')


# contains the list of all unique words in pre-trained word2vec vectors
# summarize vocabulary
modelwords = list(model.wv.vocab)
print('An example of the words in the Word2Vec model are:',modelwords[:100])

# access vector for one word
#print('\nlower',text,'\n')
    



In [ ]:
print('Lets test it!')
# lemma each lyric - maybe want to split by sentence: 
if base=='lemma':
    hap = wnl.lemmatize('happy', wn.ADJ)
elif base=='stem':
    hap = [stemmer.stemWord('happy')]  
print(base,':',hap)
print("Happy looks like this",model[hap][:10])

Now we have word2vec word embeddings - aka vectors by meaning trained on our corpus. We can now find similar words and append each of the words embedding vector to a matrix; 
apply TSNE to the matrix to project the word into a 2D space. AND plot the 2D position of the word with a label. 


#### What is Annoy?
"Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings to search for points in space that are close to a given query point. It also creates large read-only file-based data structures that are mmapped into memory so that many processes may share the same data." 

Annoy is an open source library to search for points in space that are close to a given query point. For our purpose, it is used to find similarity between words or documents in a vector space. https://markroxor.github.io/gensim/static/notebooks/annoytutorial.html

#### Lets use Annoy Indexer
Why use Annoy?
The current implementation for finding k nearest neighbors in a vector space in gensim has linear complexity via brute force in the number of indexed documents, although with extremely low constant factors. The retrieved results are exact, which is an overkill in many applications: approximate results retrieved in sub-linear time may be enough. Annoy can find approximate nearest neighbors much faster.

The Annoyindexer creates K-means clusters to be created using approximate nearest neighbors rather then the word2vec built in K-means exact method, which can be approximately 65 times faster.  

In [ ]:
#Set up the model and vector that we are using in the comparison
try:
    from gensim.similarities.index import AnnoyIndexer
except ImportError:
    raise ValueError("SKIP: Please install the annoy indexer")
    
model.init_sims()

if new_model:
    annoy_index = AnnoyIndexer(model, 500)  
    annoy_index.save('annoyindex.index')
    print('New Annoy Index Created and saved')
    
if not new_model:
    # Load the Model back from file
    print("loading indexer...")
    annoy_index = AnnoyIndexer()
    annoy_index.load('../input/affectsimilarities/annoyindex.index.index')
    annoy_index.model = model  #assign index model
    #create and save Annoy Index from a loaded `KeyedVectors` object (with 100 trees) 
    annoy_index = AnnoyIndexer(model, 500)  # number of trees does not have to match saved version, (not sure of the # tree tradeoff)   # To  get a  metric='angular' error
    print(annoy_index.model)
    print('Annoy Index Loaded')


In [ ]:
#Word2Vec.most_similar like we would traditionally, but with an added parameter, indexer. The only supported indexer in gensim as of now is Annoy.
# Derive the vector for the word "science" in our model
def getmynegh(word,exact,num):
    vector = model.wv[word][0]
    neighbors=[]
    if exact:
        # The instance of AnnoyIndexer we just created is passed 
        approximate_neighbors = model.wv.most_similar([vector], topn=num, indexer=annoy_index)

        # Neatly print the approximate_neighbors and their corresponding cosine similarity values
        print("\nApproximate Neighbors for the word ",word,":")
        for neighbor in approximate_neighbors:
            neighbors.append(neighbor)
            print('{:20}: {:.5}'.format(neighbor[0],neighbor[1]))
   
    if not exact: 
        normal_neighbors = model.wv.most_similar([vector], topn=num)
        print("\nNormal (not Annoy-indexed) Neighbors for the word ",word,":")
        for neighbor in normal_neighbors:
            neighbors.append(neighbor)
            print('{:20}: {:.5}'.format(neighbor[0],neighbor[1]))
    return(neighbors)
        
getmynegh(hap,exact=True, num=15)
getmynegh(hap,exact=False, num=15)

## Model Metrics & Initialization Time
Relationship between num_trees and initialization time:
Initialization time of the annoy indexer increases in a linear fashion with num_trees. Initialization time will vary from corpus to corpus

# If you want exact results instead of approximate cheange this line for the approxmate results line in the for statement
exact_results = [element[0] for element in model.wv.most_similar([model.wv.vectors_norm[0]], topn=100)]

x_values = []
y_values_init = []
y_values_accuracy = []

for x in range(1, 300, 10):
    x_values.append(x)
    start_time = time.time()
    annoy_index = AnnoyIndexer(model, x)
    y_values_init.append(time.time() - start_time)
    approximate_results = model.wv.most_similar([model.wv.vectors_norm[0]], topn=100, indexer=annoy_index)
    top_words = [result[0] for result in approximate_results]
    y_values_accuracy.append(len(set(top_words).intersection(exact_results)))

plt.figure(1, figsize=(12, 6))
plt.subplot(121)
plt.plot(x_values, y_values_init)
plt.title("num_trees vs initalization time")
plt.ylabel("Initialization time (s)")
plt.xlabel("num_trees")
plt.subplot(122)
plt.plot(x_values, y_values_accuracy)
plt.title("num_trees vs accuracy")
plt.ylabel("% accuracy")
plt.xlabel("num_trees")
plt.tight_layout()
plt.show()

# Model Visualization - Emotional Terms
There are multiple ways to map the multi-dimensional space of vectors onto a 2D plot.  We'll inspect 2 of them: 

1. tsne_plots
1. PCA 

But first, we need to limit our vector space to a subset of words - we only care about emotional words. While, we may want the full model to do word prediction, the maps should just be emotional terms. 

In [ ]:
# This function limits the words to be displayed by some emotwords list
def smallermodel(emotwords):
    smmodel={}
    for key, val in model.wv.vocab.items():
        if key in emotwords:
            #print('my emot is in the model:',key)
            smmodel[key] = val
    return(smmodel)

def getwords(dictmodel):
    return dictmodel.keys()



In [ ]:
# limit vectors to Russell affect terms
print(russell[:10])
crussell=stememots(russell) 
#crussell=[review_to_words(text,base) for text in russell]  #clean the 28 russel terms
print('CLEANED RUSSELL:\n',crussell)

# Use the emotional word list and find the 10 most simiar words then map those... it should give us about 200 words on our map
russmodel={}
russmodel = smallermodel(crussell)
print(type(russmodel),len(russmodel))

# limit vectors to BSM100 Labels 
clabels = stememots(ilabels)  # Labels from image classifications (SEE code block 2) 
print(print('CLEANED BSM100 LABELS:\n',clabels[:5]))

bsm100model={}
bsm100model = smallermodel(clabels)

print(type(bsm100model),len(bsm100model))


# Similarity mapping of just the affect terms
Next, we'll only keep the words in our model results that are in our wordnet affect sysnets. This is a list of 300 emotional words so it should be fairly comprehensive.   
A principal component analysis will be performed to map the relationships between the words. 

Remember our sentence... Lets use this model and see if we can do better: 
** HELP **
ummm we need to actually update the model... not sure how to do that.  I don' want to strip all the non emotional words out before I train... I feel like that would loose something. 
But I can't strip it out after because I just get a dict. umm. 

Interesting results.  I'm surprised "bang" and "chill" are in the list. 

Lets try the 28 words that James Russell originally used.  Again, we'll limit our model results to just these words.  Lets see what that looks like for comparision reasons to his model, remember not all the words will show up. Only the words that are both in our corpus and our emotional list.

The russell list is defined in <a href='#readinaffect'> affect load list section</a>

### Dimensionality reduction using PCA
PCA is a technique for reducing the number of dimensions in a dataset whilst retaining most information. It is using the correlation between some dimensions and tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed. It does not do this using guesswork but using hard mathematics and it uses something known as the eigenvalues and eigenvectors of the data-matrix. These eigenvectors of the covariance matrix have the property that they point along the major directions of variation in the data. These are the directions of maximum variation in a dataset.  (ref: https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b)

In [ ]:
# fit a 2d PCA model to the vectors, #this function creates a PCA visualization with a list of words and their simiarities as inputs
def pcavis(modelwords,modelname,ncomps):
    X = model[modelwords]
    #print(X[:5])  #prints 5 arrays of pcas
    savename='PCA and TNSE'+str(modelname)+' with '+str(ncomps)+'components.png'
    ## ADD DF.CLASS - USE THE BSM100 DATA TO CATEGORIZE EMOTIONS AS POS, NEG OR NEUTRAL USE THE CLASS AS HUE
    
    pca = PCA(n_components=ncomps)
    result = pca.fit_transform(X)  #returns the principal components, the number returned is identified in n_compontents
        
    #convert our words and the vectors into a df
    df = pd.DataFrame(list(modelwords),columns=['Words'])
    df['vec']=list(X)
    #print(df)
    #append the first 2 pca results to the df
    df['pca_one']=result[:,0]
    df['pca_two']=result[:,1]
    #df['pca_three']=result[:,2]
    #df['pca_four']=result[:,3]
    
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    
    #TNSE:
    time_start = time.time()
    tsne = TSNE(n_components=2, verbose=1, perplexity=4, n_iter=300)
    tsne_results = tsne.fit_transform(X)
    #print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
    df['tsne_one'] = tsne_results[:,0]
    df['tsne_two'] = tsne_results[:,1]
    return df

#=========================================
def mkpcaplot(df,nm,x1,x2,y1,y2):
    # Seaborn Plots
    # Define plot space
    plt.figure(figsize=(8,6))
    s1=sns.scatterplot(
        x="pca_one", y="pca_two",  #hue="y",
        palette=sns.color_palette("bright"),
        data=df,
        #legend="full",
        alpha=0.75
    )      
    #add annotation:
    for i, row in enumerate(df):
        xy = (df.pca_one[i], df.pca_two[i])
        name = df.Words[i]
        plt.annotate(name, xy=xy)
    s1.set(title='PCA First two components',
           xlim=(x1, x2),
            ylim=(y1, y2))
    plt.show()
    fname=nm+'PCA'
    plt.savefig(fname=fname,format='png')

def mktsneplot(df,nm,x1,x2,y1,y2):
    plt.figure(figsize=(8,6))

    s2=sns.scatterplot(
        x="tsne_one", y="tsne_two", #hue=y,
        palette=sns.color_palette("bright"),
        data=df,
        legend="full",
        alpha=0.3,
    )
    #add annotation:
    for i in range(1,df.shape[0]):
           s2.text(df.tsne_one[i]+0.2, df.tsne_two[i], df.Words[i], horizontalalignment='left', size='x-small', color='black')
    s2.set(title='T-SNE Plots',
            xlim=(x1, x2),
            ylim=(y1, y2))
    plt.show()
    fname=nm+'T-sne'
    plt.savefig(fname=fname,format='png')

    


In [ ]:
print('Lets visualize all of the terms in our model... be patient this takes a while')
emotmodel = smallermodel(cleanemots)  #limit the model by the cleanemots emotional word list from wordnet
print(type(emotmodel),len(emotmodel))

emotdf=pcavis(modelwords=model.wv.vocab,modelname='main',ncomps=2)  #model names: main, wordnet,  modelinput: model.wv.vocab, emotmodel, russmodel etc.
#mkplts(emotdf)
name='allwords'
namez='allwords - Zoomed'
df=emotdf
mkpcaplot(df,name,-.05,.05,-.05,.05)
mktsneplot(df,name,-30,30,-30,30)
mkpcaplot(df,namez,-.02,.02,-.02,.02)
mktsneplot(df,namez,-30,30,-30,30)

In [ ]:
#Plot Russell Affect Terms
russdf=pcavis(modelwords=russmodel, modelname='Russell 28 Terms', ncomps=2)  #model names: main, wordnet,  modelinput: model.wv.vocab, emotmodel, russmodel etc. 
name='Russell'
df=russdf
namez='Russell - Zoomed'
mkpcaplot(df,name,-.05,.05,-.05,.05)
mktsneplot(df,name,-30,30,-30,30)
mkpcaplot(df,namez,-.02,.02,-.02,.02)
mktsneplot(df,namez,-30,30,-30,30)

In [ ]:
# Plot BSM100 Affect Terms
bsmdf=pcavis(modelwords=bsm100model,modelname='Image Labels',ncomps=2)  #model names: main, wordnet,  modelinput: model.wv.vocab, emotmodel, russmodel etc. 
name='BSM100'
namez='BSM100 - Zoomed'
df=bsmdf
mkpcaplot(df,name,-.05,.05,-.05,.05)
mktsneplot(df,name,-30,30,-30,30)
mkpcaplot(df,namez,-.02,.02,-.02,.02)
mktsneplot(df,namez,-30,30,-30,30)

# Sentence Generation
Lets use 4 approachs
1. Fill in the blank
1. Synonym from wordnet 
1. Meaning from wordNet
1. Model based - using our corpus driven model
1. Markov Generator

Visually inspect the model based using displacy

In [ ]:
print('******* Reading in the classified emotion *********')
# Reload data from imageclassification by clicking add data select the Kernel output files> my work
import _pickle as cPickle
import csv
try:
    with open(r"../input/imageclassificationv2/myemtdata.pickle", "rb") as input_file:
        dat = cPickle.load(input_file)
    csvemot = pd.readcsv("../input/imageclassificationv2/myemot.csv") #read data from csv file
    print(csvemot)
    print('this data comes from my classifed emotion model:')
    classemot=dat["myemot"]
    print(classemot)
except:
    classemot='sad'
    print(classemot)

In [ ]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,300), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()
    

In [ ]:
## Get Synonyms from all of wordnet - use this to compare with model output
synonyms = [] 
antonyms = [] 
  
def getnewword(classemot):
    for syn in wn.synsets(classemot):   #for each of the synsets of our classified word
        for l in syn.lemmas():                #get the lemma
            if l.name()!= classemot:
                synonyms.append(l.name())          #append the lemma to a list
            if l.antonyms():            # Get the list of antonyms
                antonyms.append(l.antonyms()[0].name())   #append the antoynms to a list ... just because 
    return(random.choice(synonyms))

In [ ]:
# 1. READ IN MY CLASSIFIED EMOTION
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()


print('My Classified Emotion: ',classemot)
print('We need to clean it')
classemot = classemot.lower()  #lower case
classemotl =wnl.lemmatize(classemot, pos='v')  #lets lemmatize it!
classemots = stemmer.stem(classemot)
print('The lemma of our word is:',classemotl,'\nThe stem of our word is:',classemots)

In [ ]:
sen='I feel '+classemotl
print(sen)

newemot=getnewword(classemotl)

print('\n** USING wordnet synonyms **')
newsent="I feel "+ newemot
print('This uses just a synonym from wordnet:\n',newsent)

syn = wn.synsets(newemot)
elabsent="I am "+(syn[0].definition())
print('Here is an elaborate sentence:\n',elabsent)

In [ ]:
print('\nNow lets get the most similar word to our classified word using our model:')
neighbors=getmynegh(classemots,model,5) # I NEED TO ITERATE ON THE CLASSIFIED EMOTION, OR MOVE UP THE WORDNET TREE UNTIL I GET A WORD IN THE MODEL
print(type(neighbors))
smartemot=neighbors[1][0]
print(type(smartemot), smartemot)
smartsent='I feel '+ smartemot
print('\nModel Generated Sentence:\n',smartsent)

In [ ]:
#Next let’s use the displaCy library to visualize the parse tree for that sentence:

GenSent=nlp(smartsent)
print(type(GenSent))
print('Dependency Analysis')
#displacy.render(GenSent, style='dep', jupyter=True, options={'dimension':150,'compact':True})
#sentence_spans= doc.sents
#print(sentence_spans)
displacy.render(GenSent, style="dep", jupyter=True, options={'dimension':150,'compact':True})

In [ ]:
# Build the Marvovian model based off of our Corpus text.
text_model = markovify.Text(cleancorp)

print('\n\nHere are 3 randomly-generated long sentences:\n')
for i in range(3):
    print(text_model.make_sentence())
    
print('\n\nHere are 3 randomly-generated sentences of no more than 120 characters:\n')
for i in range(3):
    print(text_model.make_short_sentence(120))

## COMPARISON USING VECTORS FROM GLOVE

In [ ]:
## READ IN EMOT TERMS AND USE T-SNE PLOTS WITH PRETRAINED VECTORS IN GLOVE
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

embeddings_dict = {}
with open("../input/glove-global-vectors-for-word-representation/glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        if word in emotwords:                 #check to see if the word is an emotional word
            #print('FOUND ONE ', word)
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector

def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

tsne = TSNE(n_components=2, random_state=0)
words =  list(embeddings_dict.keys())

vectors = [embeddings_dict[word] for word in words]

Y = tsne.fit_transform(vectors)

plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")

plt.show()



In [ ]:
#zoomed in
plt.scatter(Y[:, 0], Y[:, 1])

for label, x, y in zip(words, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy=(x, y), xytext=(0, 5), textcoords="offset points")

    
plt.axis([-5,5,-5,5])

plt.show()

### Topic Modeling

# Appendix A
## TFIDF Generator 
create a instead of using our model
insiration: https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation


# number of topics to extract
n_topics = 15

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = TfidfVectorizer(max_features=5000, stop_words="english", max_df=0.95, min_df=2)
features = vec.fit_transform(emotcorpus)

from sklearn.decomposition import NMF
random_state=0
cls = NMF(n_components=n_topics, random_state=random_state)
cls.fit(features)

# list of unique words found by the vectorizer
feature_names = vec.get_feature_names()

# number of most influencing words to display per topic
n_top_words = 15

topics=[]
for i, topic_vec in enumerate(cls.components_):
    x=[]
    print(i, end=' ')
    # topic_vec.argsort() produces a new array
    # in which word_index with the least score is the
    # first array element and word_index with highest
    # score is the last array element. Then using a
    # fancy indexing [-1: -n_top_words-1:-1], we are
    # slicing the array from its end in such a way that
    # top `n_top_words` word_index with highest scores
    # are returned in desceding order
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        print(feature_names[fid], end=' ')
        x.append(feature_names[fid])
    topics.append(x)
    print()
    
emottopics=[]
# Now change each list of words into a string of words
for t in topics: 
    seperator = ' '
    emottopics.append(seperator.join(t))

for i in emots:
    print('I:',i)
    for t in emottopics:
        print('topic string:',t)
        if i in emotwords: #not sure I'm just returning my emotionaal words, it should be. 
            print('my emotion is in my topic:',t)
            emottopics.append(t)
emottopics

# Results
So this is very interesting.  First we found that the childrens boook corpus was pretty useless, as it didn't have very many of the words that we use in our language today.  I find it interesting that the words in our affect list, are located where they are.  I would have thought that laughter, pride,  and love would have been closer related.  Of course, my thinking is influenced by the various frameworks in place today.  I have a mental image of the James Russell Circumplex of Affect model with affect terms laid out in a arousal vs pleasure scale.  As Dr. Peter refered to, that we discussed in chapter 2, previous models were based on the need for studying physiology or neuroscience, not HCI.  It may be worth considering that moving forward a model based on language corpus is used for HCI needs. 

## References
https://rare-technologies.com/word2vec-tutorial/ - word2vec turtorial
https://medium.com/@shubhamagarwal328/playing-around-with-word2vec-natural-language-processing-ccd10a044b1 - word2vec

word2vec explained
https://israelg99.github.io/2017-03-23-Word2Vec-Explained/

moreW2V - w1-w2 simialarity stuff < - READ MORE
https://www.freecodecamp.org/news/how-to-get-started-with-word2vec-and-then-how-to-make-it-work-d0a2fca9dad3/

YEP USED THIS TOO
http://www.pitt.edu/~naraehan/presentation/word2vec-try.html

NLP REFERENCES
https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk

MAYBE USEFUL
http://www.lumenai.fr/blog/quick-review-on-text-clustering-and-text-similarity-approaches

AH THE ANNOY MODEL 
https://markroxor.github.io/gensim/static/notebooks/annoytutorial.html

OH CAN I USE THE MARKOVIAN CHAIN <- DO MORE HERE
https://towardsdatascience.com/nlg-for-fun-automated-headlines-generator-6d0459f9588f


TOPIC MODELING
https://sanjayasubedi.com.np/nlp/nlp-with-python-topic-modeling/



# TO DO - COOL STUFF

DO THAT LANGUAGE MAPPING - WORD TYPE STUFF

# Now use SpaCy to do some NLP exploration
corpus=getcorp()

doc = nlp(corpus[:200000])

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

cols = ("Word", "Lemma","Tag"," POS","Tag - Long", "Stopword") 
rows = []




print('\nSending the selected corpus through spacy we get the Lemma, Part of Speech, and Stopword')

cor_pos = getpos(doc)
cor_pos[:10]


Lets do a language map of our sentence to make sure it has all the sentence parts